In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
!pip install python-dotenv openAI langchain_core langchain langchain_openai langchain_chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.0 MB/s eta 0:00:00


In [ ]:
# 환경변수 설정

In [5]:
# 라이브러리 불러오기
from dotenv import load_dotenv
import os
from langchain_openai import OpenAI

In [6]:
# .env 파일에서 환경 변수 로드
load_dotenv("/content/drive/MyDrive/Colab Notebooks/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")
# 오픈AI 대규모 언어 모델 초기화
llm = OpenAI()

In [7]:
# 예제 1-10 <문자열 프롬프트 템플릿>

# 라이브러리 불러오기
from langchain_core.prompts import PromptTemplate
# 주어진 주제에 대한 농담을 요청하는 프롬프트 템플릿 정의
prompt_template = PromptTemplate.from_template("주제 {topic}에 대해 금융 관련 짧은 조언을 해주세요")
# '투자' 주제로 프롬프트 템플릿 호출
prompt_template.invoke({"topic": "투자"})

StringPromptValue(text='주제 투자에 대해 금융 관련 짧은 조언을 해주세요')

In [8]:
# 예제 1-11 <챗 프롬프트 템플릿>

# 라이브러리 불러오기
from langchain_core.prompts import ChatPromptTemplate

# 챗 프롬프트 템플릿 정의: 사용자와 시스템 간의 메시지를 포함
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 금융 조언가입니다."),
    ("user", "주제 {topic}에 대해 금융 관련 조언을 해주세요")
])
# '주식' 주제로 챗 프롬프트 템플릿 호출
prompt_template.invoke({"topic": "주식"})


ChatPromptValue(messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='주제 주식에 대해 금융 관련 조언을 해주세요', additional_kwargs={}, response_metadata={})])

In [9]:
# 예제 1-12 <메시지 자리 표시자 템플릿>

# 라이브러리 불러오기
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

# (방법1) 메시지 자리 표시자를 포함한 챗 프롬프트 템플릿 정의
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 금융 조언가입니다."),
    MessagesPlaceholder("msgs")
])
# 메시지 리스트를 'msgs' 자리 표시자에 전달하여 호출
prompt_template.invoke({"msgs": [HumanMessage(content="안녕하세요!")]})


ChatPromptValue(messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요!', additional_kwargs={}, response_metadata={})])

In [10]:
# (방법2) MessagesPlaceholder 클래스를 명시적으로 사용하지 않고도 비슷한 작업을 수행할 수 있는 방법
prompt_template = ChatPromptTemplate.from_messages([
("system", "당신은 유능한 금융 조언가입니다."),
("placeholder", "{msgs}") # <-- 여기서 'msgs'가 자리 표시자로 사용됩니다.
])
# 메시지 리스트를 'msgs' 자리 표시자에 전달하여 호출
prompt_template.invoke({"msgs": [HumanMessage(content="안녕하세요!")]})


ChatPromptValue(messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요!', additional_kwargs={}, response_metadata={})])

In [13]:
# 예제 1-13 <`PromptTemplate`를 이용한 퓨샷 프롬프트>

# 라이브러리 불러오기
from langchain_core.prompts import PromptTemplate
# 질문과 답변을 포맷하는 프롬프트 템플릿 정의
example_prompt = PromptTemplate.from_template("질문: {question}\n답변: {answer}")

examples = [
    {
        "question": "주식 투자와 예금 중 어느 것이 더 수익률이 높은가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자
""",
    } ,
    {
        "question": "부동산과 채권 중 어느 것이 더 안정적인 투자처인가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권
""",
    },
]

print(example_prompt.invoke(examples[0]).to_string())

질문: 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자



In [14]:
# 예제 1-14 <`FewShotPromptTemplate` 를 이용한 퓨샷 프롬프트>

# 라이브러리 불러오기
from langchain_core.prompts import FewShotPromptTemplate
# FewShotPromptTemplate 생성
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="질문: {input}",
    input_variables=["input"],
)
# '주식 투자' 주제로 프롬프트 호출 및 출력
print(
    prompt.invoke({"input": "부동산 투자의 장점은 무엇인가?"}).to_string()
)

질문: 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자


질문: 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권


질문: 부동산 투자의 장점은 무엇인가?


In [18]:
# 예제 1-15 <예제 선택기를 이용한 퓨샷 프롬프트>

# 라이브러리 불러오기
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

# 예제 선택기 초기화
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,  # 사용할 예제 목록
    OpenAIEmbeddings(api_key=api_key),  # 임베딩 생성에 사용되는 클래스
    Chroma,  # 임베딩을 저장하고 유사도 검색을 수행하는 벡터 스토어 클래스
    k=1,  # 선택할 예제의 수
)

# 입력과 가장 유사한 예제 선택
question = "부동산 투자의 장점은 무엇인가?"
selected_examples = example_selector.select_examples({"question": question})

# 선택된 예제 출력
print(f"입력과 가장 유사한 예제: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")


입력과 가장 유사한 예제: 부동산 투자의 장점은 무엇인가?


answer: 
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권

question: 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?


In [23]:
# 예제 1-16 <퓨샷 프롬프트 AI 모델 적용>

# 라이브러리 불러오기
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

# 예제 프롬프트 템플릿 생성
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="질문: {question}\n답변: {answer}"
)

# 퓨샷 프롬프트 템플릿 설정
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="다음은 금융 관련 질문과 답변의 예시입니다:",
    suffix="질문: {input}\n답변:",
    input_variables=["input"]
)

In [24]:
# AI 모델 설정
model = ChatOpenAI(model_name="gpt-3.5-turbo")

In [27]:
# 체인 구성 및 실행
chain = prompt | model  # RunnableSequence를 사용하여 체인 연결

response = chain.invoke({"input": "부동산 투자의 장점은 무엇인가?"})  # invoke 메서드 사용
print(response.content)

부동산 투자의 장점은 안정적인 수익과 가치의 상승이 있습니다. 또한 임대 수입을 얻을 수 있고 부동산 시장이 안정적이기 때문에 장기적으로 안정적인 수익을 기대할 수 있습니다.


In [31]:
# 예제 1-17 <랭체인 허브의 특정 프롬프트 불러오기>

from langchain import hub
# 최신 버전의 프롬프트 불러오기
prompt = hub.pull("rlm/rag-prompt")
# 특정 버전의 프롬프트 불러오기
hub.pull("rlm/rag-prompt:c9839f14")


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': 'c9839f14c19d5274f21842ead918a907f88e255a96d3badf8039a4fbfeab24bd'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])